In [1]:
import cv2
import imageio
import json
from matplotlib import pyplot as plt
import numpy as np
import os
from PIL import Image
from skimage.transform import resize
import sys
import tarfile
# from torchvision import transforms
# import torch
import xml.etree.ElementTree as ET
from zipfile import ZipFile

In [3]:
code_folder = os.getcwd()
annotation_folder = os.path.join(code_folder,'../../dataset/Stanford/Annotation')
images_folder = os.path.join(code_folder,'../../dataset/Stanford/Images')

In [5]:
size_x = []
size_y = []

for breed in sorted(os.listdir(annotation_folder)):
  for dog in sorted(os.listdir(os.path.join(annotation_folder, breed))):
    with open(os.path.join(annotation_folder, breed, dog), 'r') as f:
      tree = ET.parse(f)
      root = tree.getroot()
      size_x.append(int(root.find('size/width').text))
      size_y.append(int(root.find('size/height').text))

In [8]:
intra_breed_error = {}
breed_error_mean_values = []

def calc_ssq_diff(a,b):
  ssq = np.sum(np.square(a-b))
  a2 = np.sum(np.square(a))
  b2 = np.sum(np.square(b))
  return ssq / np.sqrt(a2 * b2)

def calc_diff_intra_breed(a):
  result = np.zeros((a.shape[0],a.shape[0]))
  for i in range(a.shape[0]):
    for j in range(i+1, a.shape[0], 1):
      result[i][j] = calc_ssq_diff(a[i],a[j])
  return result

def calc_diff_numpy_intra_breed(a):
  diff_squared = np.square(a[:,np.newaxis,:,:,:]-a[np.newaxis,:,:,:,:])
  ssq = np.sum(diff_squared, axis=(2,3,4))
  a_sq_sum = np.sum(np.square(a), axis=(1,2,3))
  normalizing_terms = np.sqrt(a_sq_sum[:,np.newaxis] * a_sq_sum[np.newaxis,:])
  normalized_ssq = ssq / normalizing_terms
  return normalized_ssq

In [9]:
for breed in sorted(os.listdir(images_folder)):
  breed_imgs = np.zeros((len(os.listdir(os.path.join(images_folder, breed))), 500, 500, 3))
  i = 0
  for dog in sorted(os.listdir(os.path.join(images_folder, breed))):
    img = np.array(imageio.imread(os.path.join(images_folder, breed, dog)))
    breed_imgs[i] = np.array(resize(img, (500,500,3)))
    i += 1
  # intra_breed_error[breed] = calc_diff_intra_breed(breed_imgs) 
  intra_breed_error[breed] = calc_diff_numpy_intra_breed(breed_imgs)
  error_values = intra_breed_error[breed][np.triu_indices(intra_breed_error[breed].shape[0], k = 0)]
  fig, ax = plt.subplots()
  ax.boxplot(error_values, 0, 'x', 0)
  ax.set_title(breed)
  plt.savefig(os.path.join(code_folder, '../../tmp/', 'Error_plots', breed + '.png'))
  plt.close()
  breed_error_mean_values.append(intra_breed_error[breed].mean())

fig, ax = plt.subplots()
ax.boxplot(breed_error_mean_values, 0, 'x', 0)
ax.set_title("Average Error Values")
plt.savefig(code_folder, '../../tmp/', 'Error_plots','avgError.png')
plt.close()

MemoryError: Unable to allocate 129. GiB for an array with shape (152, 152, 500, 500, 3) and data type float64